# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [81]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import os
import json
import time
import folium


# Import API key

from api_keys import geoapify_key




In [82]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()




,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,mataura,-46.1927,168.8643,8.53,64,86,1.18,NZ,1680818293
1,1,katangli,51.7102,143.2326,-1.38,90,100,6.97,RU,1680818293
2,2,kamenka,51.3223,42.7678,10.46,68,100,5.14,RU,1680818293
3,3,bluff,-46.6000,168.3333,9.86,67,65,3.94,NZ,1680818294
4,4,ushuaia,-54.8000,-68.3000,11.81,43,75,7.72,AR,1680818189


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [83]:

# Define the base URL for the Geoapify API 
base_url = "https://api.geoapify.com/v2/places"

# Create a folium map object.
map = folium.Map(location=[0, 0], zoom_start=2)

# Iterate over the rows in the city_data_df DataFrame.
for index, row in city_data_df.iterrows():
    # Create a folium marker for each city.
    marker = folium.CircleMarker(
        location=[row["Lat"], row["Lng"]],
        radius=row["Humidity"],
        color="red",
        fill=True,
        fill_color="red",
        fill_opacity=0.1,
        tooltip=f"{row['City']}, {row['Country']}: {row['Humidity']}% humidity"
    )
    # Add the marker to the folium map object.
    marker.add_to(map)

# Display the map.
map






### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [84]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_df = city_data_df[(city_data_df["Max Temp"] > 20) & (city_data_df["Max Temp"] < 25) & (city_data_df["Humidity"] > 50) & (city_data_df["Humidity"] < 70)]

# Drop any rows with null values
ideal_weather_df = ideal_weather_df.dropna()

# Display sample data
ideal_weather_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
6,6,carnarvon,-24.8667,113.6333,22.30,69,26,6.95,AU,1680818294
16,16,rikitea,-23.1203,-134.9692,24.96,66,46,4.16,PF,1680818297
44,44,umm lajj,25.0213,37.2685,22.97,66,1,3.51,SA,1680818303
63,63,lompoc,34.6391,-120.4579,22.08,56,20,4.63,US,1680818307
138,138,santiago del estero,-27.7951,-64.2615,24.55,64,53,1.58,AR,1680818321


### Step 3: Create a new DataFrame called `hotel_df`.

In [85]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df = hotel_df.assign(Hotel_Name="")

# Display sample data
hotel_df.head()


,City,Country,Lat,Lng,Humidity,Hotel_Name
6,carnarvon,AU,-24.8667,113.6333,69,
16,rikitea,PF,-23.1203,-134.9692,66,
44,umm lajj,SA,25.0213,37.2685,66,
63,lompoc,US,34.6391,-120.4579,56,
138,santiago del estero,AR,-27.7951,-64.2615,64,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [86]:
# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    
    # Get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    radius = 10000
    params["filter"] = f"circle:{radius}@{latitude},{longitude}"
    params["bias"] = f"proximity:{latitude},{longitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address =  name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df



carnarvon - nearest hotel: No hotel found
rikitea - nearest hotel: No hotel found
umm lajj - nearest hotel: No hotel found
lompoc - nearest hotel: No hotel found
santiago del estero - nearest hotel: No hotel found
pasni - nearest hotel: No hotel found
sur - nearest hotel: No hotel found
mount isa - nearest hotel: No hotel found
half moon bay - nearest hotel: No hotel found
vao - nearest hotel: No hotel found
swellendam - nearest hotel: No hotel found
tacuarembo - nearest hotel: No hotel found
viedma - nearest hotel: No hotel found
necochea - nearest hotel: No hotel found
correntina - nearest hotel: No hotel found
dwarka - nearest hotel: No hotel found
khorramshahr - nearest hotel: No hotel found
kloulklubed - nearest hotel: No hotel found
todos santos - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel_Name,Hotel Name
6,carnarvon,AU,-24.8667,113.6333,69,,No hotel found
16,rikitea,PF,-23.1203,-134.9692,66,,No hotel found
44,umm lajj,SA,25.0213,37.2685,66,,No hotel found
63,lompoc,US,34.6391,-120.4579,56,,No hotel found
138,santiago del estero,AR,-27.7951,-64.2615,64,,No hotel found
192,pasni,PK,25.2631,63.4710,65,,No hotel found
196,sur,OM,22.5667,59.5289,62,,No hotel found
214,mount isa,AU,-20.7333,139.5000,68,,No hotel found
222,half moon bay,US,37.4636,-122.4286,57,,No hotel found
257,vao,NC,-22.6667,167.4833,68,,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.